This program produces a Shiny app that produces a few different visualizations of a random walk on the N-cycle. 

In [ ]:
library(shiny)
library(igraph)

ui <- navbarPage("Random Walk on the N Cycle",
    tabPanel("Version 1",
        h4("This program simulates a random walk on a cyclical graph with a specified number of vertices.
           At each step, the walk moves counterclockwise with some probability p."),
        h4("Specify the number of vertices and the value of p. Then press 'Begin Simulation' to begin
           and 'Take Step' to take a step."),
        br(),
        sidebarLayout(
            sidebarPanel(
                numericInput("n", "Number of vertices:",
                          value = 6),
                numericInput("p", "Probability of moving counterclockwise:",
                                  value = 0.5),
                actionButton("start", "Begin Simulation"),
                actionButton("step", "Take Step")
            ),
            mainPanel(
                plotOutput("plotCycle"),
                textOutput("printStep"),
                plotOutput("plotStep")
            )
        )     
    ),
    tabPanel("Version 2",
        h4("This program simulates a random walk on the n-cycle for a specified number of steps."),
        h4("It produces a graph of the proportion of time the walk spent at each state and a plot of
            the total variation distance between the walk and the uniform distribution."),
        h4("Specify the number of vertices, the probability of moving up (or from n to 1), and the
           number of steps, and press 'Run' to run the simulation."),
        br(), 
        br(),
        br(),
        sidebarLayout(
            sidebarPanel(
                numericInput("num",
                             "Number of vertices:",
                             value = 6),
                numericInput("prob",
                             "Probability of moving up:",
                             value=0.5),
                numericInput("s",
                             "Number of steps:",
                             value = 200),
                actionButton("run", "Run")
            ),
            mainPanel(
                plotOutput("ncycle"),
                textOutput("totalVar")
            )
        )
    )
)

# Define server logic required to draw a histogram
server <- function(input, output) {
    ############## VERSION 1 #################
    
    # Vertices = vector of the states (numbers from 1 to n)
    # Visits = vector of the number of visits to each state
    # curr = the state we're currently on (will be set to a random state once the start button is clicked)
    # time = the step we're currently on
    data <- reactiveValues(Vertices=c(), Visits=c(), curr=1, time=0, g=graph(edges=c(), n=1, dir=FALSE))
    
    ## create an n-cycle graph
    makecycle <- function(n) {
        Connections <- c(n,1)
        for (i in 1:(n-1)) {
            Connections <- c(Connections, i,i+1)
        }
        a <- graph(edges=Connections, n=n, dir=FALSE)
        return(a)
    }
    
    ## when the start button is clicked, initialize the data according to the inputted parameters
    begin <- observeEvent(input$start, {
        data$Vertices <- c(1:input$n)
        data$Visits <- rep(0, input$n) 
        data$curr <- floor(runif(1)*input$n) + 1 #start at random state between 1 and n
        data$time <- 0
        data$g <- makecycle(input$n)
        V(data$g)$color <- "white"
    })
    
    ## when the step button is clicked, update the data
    step <- observeEvent(input$step, {
        if (runif(1) < input$p) { #with probability p
            if (data$curr != input$n) { #if the current state is not n
                data$curr <- data$curr + 1 #move up 1
            } else { #if the current state is n
                data$curr <- 1 #move to 1
            }
        } else { #with probability 1-p
            if (data$curr != 1) { #if the current state is not 1
                data$curr <- data$curr - 1 #move down 1
            } else { #if the current state is 1
                data$curr <- input$n #move to n
            }
        }
        data$Visits[data$curr] <- data$Visits[data$curr] + 1
        V(data$g)$color <- ifelse(V(data$g)==data$curr,"green","white")
        data$time <- data$time + 1
    })
    
    ## Make the bar plot
    output$plotStep <- renderPlot({
        if (input$start) {
            barplot(data$Visits, names=data$Vertices, xlab="State", ylab="Number of visits",
                    main=paste("Random Walk on the", input$n, "cycle, p =", input$p),
                    ylim=c(0, max(data$Visits)+1))
        }
    })
    
    ## Make the n cycle graph
    output$plotCycle <- renderPlot({
        if (input$start) {
            plot(data$g, vertex.size=18, vertex.color=V(data$g)$color, edge.color="black",
                 edge.width=2, layout=layout_in_circle(data$g, order = V(data$g)))
        }
    })
    
    ## Print which step we're on
    output$printStep <- renderText({
        if (input$start) {
            paste0("Step: ", data$time)
        }
    })
    
    
    ########## VERSION 2 #############
    ## Make the plot of the simulation
    output$ncycle <- renderPlot({
        runwalk()
    })
    
    ## when the run button is clicked, run the ncycle function with the inputted parameters
    runwalk <- eventReactive(input$run, {
        ncycle(input$num, input$prob, input$s)
    })
    
    ## print the total variation
    output$totalVar <- renderText({
        paste("Total variation distance from the uniform distribution:", round(runwalk(), digits=3))
    })
    
    ## Function for running a random walk on the n cycle
    ncycle <- function(n, p, s) {
        numSteps <- rep(0, n) #vector of the number of visits to each state (initialized to 0 for all states)
        Proportions <- rep(0, n) #vector of the proportion of time spent at each state
        x = floor(runif(1)*n + 1) #starting state: random integer between 1 and n
        step <- 0 #counter of which step we're on, starts at 0
        TotalVariation <- c() #will be a vector of the total variation from uniform at each step
        ## run the walk for s steps
        for (i in 1:s) {
            if (runif(1) < p) { #with probability p
                if (x != n) { #if the current state is not n
                    x <- x + 1 #move up 1
                } else { #if the current state is n
                    x <- 1 #move to 1
                }
            } else { #with probability 1-p
                if (x != 1) { #if the current state is not 1
                    x <- x - 1 #move down 1
                } else { #if the current state is 1
                    x <- n #move to n
                }
            }
            numSteps[x] <- numSteps[x] + 1 #record the visit to the new state
            step <- step + 1 #increase the step counter
            # update the proportions
            for (i in 1:n) {
                Proportions[i] <- numSteps[i] / step
            }
            # calculate total variation from uniform distribution for this step
            currTV <- 0 
            for (i in 1:n) {
                currTV <- currTV + abs(Proportions[i] - 1/n)
            }
            currTV <- 0.5*currTV
            TotalVariation <- c(TotalVariation, currTV) #add this step's total variation to the vector 
        } 
        # Proportions vector, but with each entry rounded to 3 decimal places
        ProportionsRounded <- Proportions 
        for (i in 1:n) {
            ProportionsRounded[i] = round(Proportions[i], digits=3)
        }
        finalTV <- TotalVariation[s] #total variation at the final step
        # create the graphs
        par(mfrow=c(1,2), oma = c(0, 0, 2, 0)) #split viewer window so we can display two graphs at once
        
        # Bar graph of the proportion of time spent at each state
        cycleplot <- barplot(ProportionsRounded, names=c(1:n), xlab="State", 
                             ylab="Proportion of steps", main="Time at Each State")
        text(x = cycleplot, y = ProportionsRounded, label = ProportionsRounded, pos=1, cex = 0.8, col = "black")
        abline(h=1/n, col="blue", lty=2) #reference line for the uniform distribution
        
        # Plot of the total variation at each step
        plot(TotalVariation, type="l", xlab="Step", ylab="Total variation distance from uniform",
             main="Total Variation")
        mtext(paste("Random Walk on the", n, "cycle for", s, "steps, p =", p), outer=TRUE, cex=1.5)
        return(finalTV) #return final total variation
    }
}


shinyApp(ui=ui, server=server)